# E5.2 - Building ion current models from data 

In this exercise we will explore some of the challenges that come with optimizing a model to data. We'll use a moderately complex model for the ultra-rapidly activating delayed rectifier current ($I_{\rm{Kur}}$). This model is simple in the sense that it has identical rate constants for each activating and deactivating transition, and it is linear. However, you'll see that there are still 12 parameters that can be varied during the fitting process, which can make for challenges in hand tuning the model to a simple activation curve.

We are using a 5-state Markov model published by Zhou et al. 2012:

<img src="fig/Model.svg" width=400>

Where: $$\alpha{} = e^{\frac{V-P1}{P2}}$$.
$$\beta{} = \frac{e^{\frac{V-P3}{P4}}\cdot{}e^{\frac{-(V-P5)}{P6}}}{P7+P8\cdot{}e^{\frac{-(V-P9)}{P10}}}$$.
$$ K_1 = const_1 $$
$$ K_2 = const_2 $$

Below we will take some time to try to fit this model just to some activation data for this current. In this case we have also allowed the peak conductance ($g_K$) to be a free parameter for your tuning. See how close you can get to the experimental data.

In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import scipy.optimize as opt
import math
from ObFunc import f, Activation

In [2]:
from ipywidgets import interact, FloatSlider
import K_widget as K
import importlib

In [3]:
importlib.reload(K)
K.Markov_Widget()

You can probably see that this model is underdetermined and that there are far more parameters than necessary or helpful for fitting this activation curve. What do you think is the reason for this?

### Combinatorial optimisation
Next we will employ a commonly-used algorithm (the Nelder-Mead simplex, or AMOEBA) to optimise the parameter set, and see if it can beat you for accuracy. This algorithm is a form of derivative-free approach to optimisation our voltage clamp activation function: 

First let's specify the experimental data you will use for fitting:

In [4]:
SSA_data = np.loadtxt("SS.txt",dtype='float')

V = SSA_data[:,0] # voltage
I = SSA_data[:,1] # current

Next we have to define a cost function for comparing the model generated data and the experimental data we have imported. See if you can create this function from the inputs and outputs prescribed below:

In [5]:
def cost(P,V,I,duration):
    
    P = P.tolist() # Opt.fmin returns an array every iteration...
                    # but we pass a list to Activation, so we need to typeset.
    outs = Activation(P,V,duration)
    model_peaks = outs['I_peak']
       
    dev_vector = model_peaks - I
    error = np.linalg.norm(dev_vector,2) # calculating the error as the L2 norm
    
    return error

Run the optimization and see how it performs:

In [ ]:
step_length = 1000
init_params = [45,20,65,50,20,15,1,0.02,29,15,1e-5,1e-5,0.5]#insert your parameter set as a starting point

[P_opt, f_opt, iters, funcalls, warnflag] = opt.fmin(cost, init_params, args=(V,I,step_length), maxiter = 300, maxfun = 300, full_output=True, disp=True)


Finally plot your optimized data compared to the experimental data below. Did the algorithm do better than your hand tuning?

In [ ]:
# Your code here:

P_opt = P_opt.tolist()
dats = Activation(P_opt,V,I,step_length)
opt_model_I = dats['I_peak']

plt.figure()
plt.plot(V,I,'b-')
plt.plot(V,opt_model_I,'r-')
plt.xlabel('Step potential [mV]')
plt.ylabel('Current (A/F)')